In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [2]:
%load_ext dotenv
%dotenv

In [3]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-06-01",
    temperature=0.7
)


In [4]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="{input}")
])

In [5]:
output_parser = StrOutputParser()

In [11]:
from langchain_core.runnables import RunnableLambda

# Runnable Lambda を使うことで、任意の関数をチェーンに組み込むことができる
# 独自の処理を chain に組み込みたい場合にはよく使う
def upper(text: str) -> str:
    return text.upper()

In [7]:
chain = prompt | llm | output_parser | RunnableLambda(upper)

In [8]:
output = chain.invoke({"input":"What is the capital of France?"})

In [9]:
output

"HELLO! IT SEEMS LIKE YOUR MESSAGE IS MISSING SOME CONTEXT OR CONTENT. COULD YOU CLARIFY OR PROVIDE MORE DETAILS ABOUT WHAT YOU'RE LOOKING FOR? I'M HERE TO HELP!"

In [12]:
# @chain デコレーターを使うことも可能
from langchain_core.runnables import chain
@chain
def upper(text: str) -> str:
    return text.upper()

chain_upper = prompt | llm | output_parser | upper
output = chain_upper.invoke({"input":"What is the capital of France?"})

print(output)

IT SEEMS LIKE YOUR MESSAGE MIGHT BE INCOMPLETE OR MISSING SOME DETAILS. COULD YOU CLARIFY OR PROVIDE MORE INFORMATION SO I CAN ASSIST YOU BETTER?


In [13]:
# 実は、RunnableLambda()や@chain を使わなくても | の接続先が Runnable の場合は自動で変換される
def upper2(text:str) -> str:
    return text.upper()

chain_upper2 = prompt | llm | output_parser | upper2
output = chain_upper2.invoke({"input":"What is the capital of France?"})

print(output)

HELLO! IT SEEMS LIKE YOUR MESSAGE IS EMPTY OR INCOMPLETE. COULD YOU CLARIFY OR PROVIDE MORE DETAILS ABOUT WHAT YOU NEED HELP WITH?


In [14]:
# Runnable の入出力の型は合わせる必要がある
chain_error = prompt | llm | upper
# これはエラーになる
chain_error.invoke({"input":"What is the capital of France?"})

AttributeError: 'AIMessage' object has no attribute 'upper'

In [22]:
def upper3(msg:AIMessage) -> str:
    return msg.content.upper()

chain_msg = prompt | llm | upper3
print(chain_msg.invoke({"input":"What is the capital of France?"}))

HELLO! IT SEEMS LIKE YOUR MESSAGE MIGHT BE INCOMPLETE OR UNCLEAR. COULD YOU PROVIDE MORE DETAILS OR CLARIFY WHAT YOU'RE LOOKING FOR? I'M HERE TO HELP!
